In [ ]:
import math
import dask.config
import dask.array as da
import distributed
from distributed.diagnostics import MemorySampler

n_gib = 16
side = int(math.sqrt(n_gib * 2) * 2**13)
a = da.random.random((side, side))
b = (a @ a.T).sum()

ms = MemorySampler()

configs = {
    f"{n_gib} GiB,no shm,dask spill": ({}, "8 GiB"),
    f"{n_gib} GiB,no shm,OS swap": ({}, False),
    f"{n_gib} GiB,shm,target=0.6": (
        {
            "temporary-directory": "/home/crusaderky/shm",
            "distributed.worker.memory.shared": True,
        },
        "8 GiB",
    ),
    f"{n_gib} GiB,shm,target=0.01": (
        {
            "temporary-directory": "/home/crusaderky/shm",
            "distributed.worker.memory.shared": True,
            "distributed.worker.memory.target": 0.01,
        },
        "8 GiB",
    ),
}
for title, (cfg, memory_limit) in configs.items():
    print(title)
    with dask.config.set(cfg):
        with distributed.Client(
            n_workers=8, memory_limit=memory_limit, threads_per_worker=2
        ):
            with ms.sample(title):
                b.compute()
    print("done")
    if "dask spill" in title:
        input()

In [ ]:
ms.plot(kind="area-h", grid=True, figsize=(40, 15))